In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline 

In [1]:
from yaml import load as yaml_load
import os
import sys
import json
sys.path.append('../')
import findspark
findspark.init()
import pyspark
from pyspark.sql.types import LongType
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler, VectorAssembler, VectorIndexer, StandardScaler
from pyspark.sql.functions import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, udf
from pyspark.sql.types import LongType, StringType, FloatType, IntegerType, DoubleType
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
import re
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml import Pipeline
spark = SparkSession.builder.master("local[3]").appName("Data cleaning").getOrCreate()

In [2]:
from src.data.make_dataset import LoadDataframe

In [3]:
def _load_config_file(config_file):
    """
    Load configuration file
    :param config_file: is the configuration file
    :return: configuration
    :rtype: dict
    """
    with open(config_file) as yml_config:
        return yaml_load(yml_config)

def _build_configuration(config_file):
    """
    Build the operation configuration dict
    :param config_file: is the path to the yaml config_file
    :type: string
    :return: config: global configuration
    :rtype dict
    """
    # yaml config
    config = _load_config_file(config_file)
    return config
def visualisation_prediction(y_test, y_pred):
    import matplotlib
    import matplotlib.pyplot as plt
    matplotlib.rc('xtick', labelsize=30) 
    matplotlib.rc('ytick', labelsize=30) 
    fig, ax = plt.subplots(figsize=(50, 40))
    plt.style.use('ggplot')
    plt.plot(y_pred, y_test, 'ro')
    plt.xlabel('Predicted Crime', fontsize = 30)
    plt.ylabel('Actual Crime', fontsize = 30)
    plt.title('Predicted Y (Crimes) to the Actual Y (Crimes)', fontsize = 30)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
    
    
def duration_day_func(x):
    """
    :return:
    """
    from astral import Astral
    city_name = 'Chicago'
    a = Astral()
    a.solar_depression = 'civil'
    city = a[city_name]
    sun = city.sun(date=x, local=True)
    return float((sun['sunset'] - sun['sunrise']).total_seconds())

extract_blok = udf(lambda x : re.findall(r"(\w+)$", x)[0], StringType())
isStreet = udf(lambda x :  1 if x in ['ST', 'St', 'st'] else 0, IntegerType())
isAV = udf(lambda x : 1 if x in ['Ave', 'AV', 'AVE'] else 0, IntegerType())
isBLVD = udf(lambda x : 1 if x in ['BLVD'] else 0, IntegerType())
isRD = udf(lambda x : 1 if x in ['RD'] else 0, IntegerType())
isPL = udf(lambda x : 1 if x in ['PL', 'pl'] else 0, IntegerType())
isBROADWAY = udf(lambda x : 1 if x in ['BROADWAY', 'Broadway'] else 0, IntegerType())
isPKWY = udf(lambda x : 1 if x in ['PKWY', 'Pkwy'] else 0, IntegerType())
duration_day_udf = udf(lambda x :   duration_day_func(x), FloatType())

config_file = "/home/ml/Documents/crimes_chigaco/config/config.yml"
config = _build_configuration(config_file)

In [4]:
%%time
obj_df_loaded = LoadDataframe(config, '2013', '2014')

CPU times: user 891 µs, sys: 97 µs, total: 988 µs
Wall time: 998 µs


In [5]:
df_temp = obj_df_loaded.df_temperature()
df_sky  = obj_df_loaded.df_sky()
df_socio = obj_df_loaded.df_socio()

In [6]:
from pyspark.sql.functions import isnan, when, count, col, isnull

In [7]:
features_socio = ['pct_housing_crowded',
 'pct_households_below_poverty','pct_age16_unemployed',
 'pct_age25_no_highschool',
 'pct_not_working_age',
 'per_capita_income',
 'hardship_index']
for f in features_socio:
    df_socio = df_socio.withColumn(f, df_socio[f].cast(FloatType()))

In [8]:
df_crime = obj_df_loaded.df_crime()
df_crime_socio = df_crime.join(df_socio, ['community_area_number'], "inner")

/usr/lib/spark/spark-2.4.0-bin-hadoop2.7/python/pyspark/sql/dataframe.py:1793: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [9]:
df_crime_socio = df_crime_socio.na.drop()

In [10]:
df_crime_socio = df_crime_socio.withColumn("block_extract", extract_blok(df_crime_socio.block))

In [11]:
df_crime_socio = (df_crime_socio.
                  withColumn("isStreet",isStreet(df_crime_socio.block_extract)).
                  withColumn("isAV", isAV(df_crime_socio.block_extract)).
                  withColumn("isBLVD", isBLVD(df_crime_socio.block_extract)).
                  withColumn("isRD", isRD(df_crime_socio.block_extract)).
                  withColumn("isPL", isPL(df_crime_socio.block_extract)).
                  withColumn("isBROADWAY", isBROADWAY(df_crime_socio.block_extract)).
                  withColumn("isPKWY", isPKWY(df_crime_socio.block_extract)))

In [12]:
df_crime_socio = df_crime_socio.withColumn('duree_day', duration_day_udf('date'))

In [13]:
df_crime_socio.printSchema()

root
 |-- community_area_number: string (nullable = true)
 |-- id: string (nullable = true)
 |-- cas_number: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- block: string (nullable = true)
 |-- iucr: string (nullable = true)
 |-- primary_type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location_description: string (nullable = true)
 |-- arrest: string (nullable = true)
 |-- domestic: string (nullable = true)
 |-- beat: string (nullable = true)
 |-- district: string (nullable = true)
 |-- ward: string (nullable = true)
 |-- fbi_code: string (nullable = true)
 |-- x_coordinate: string (nullable = true)
 |-- y_coordinate: string (nullable = true)
 |-- year: string (nullable = true)
 |-- updated_on: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- location: string (nullable = true)
 |-- community_area_name: string (nullable = true)
 |-- pct_housing_crowded: float (nullable = tru

In [14]:
df_crime_socio.limit(1000).toPandas().sample(5)

,community_area_number,id,cas_number,date,block,iucr,primary_type,description,location_description,arrest,...,hardship_index,block_extract,isStreet,isAV,isBLVD,isRD,isPL,isBROADWAY,isPKWY,duree_day
962,67,8951097,HW100137,2013-01-01 01:58:00,065XX S LAFLIN ST,0496,ASSAULT_BATTERY,AGGRAVATED DOMESTIC BATTERY: KNIFE/CUTTING INST,RESIDENCE,false,...,89.0,ST,1,0,0,0,0,0,0,33110.0
293,24,10272489,HY460153,2013-01-01 12:00:00,016XX N FRANCISCO AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,APARTMENT,false,...,10.0,AVE,0,1,0,0,0,0,0,33129.0
823,68,8950888,HW100069,2013-01-01 00:43:00,056XX S UNION AVE,0486,ASSAULT_BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,true,...,94.0,AVE,0,1,0,0,0,0,0,33108.0
731,1,21171,HW549623,2013-11-27 03:12:00,075XX N GREENVIEW AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,false,...,39.0,AVE,0,1,0,0,0,0,0,34094.0
141,49,10085466,HY274713,2013-10-23 12:00:00,113XX S EMERALD AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,RESIDENCE,false,...,52.0,AVE,0,1,0,0,0,0,0,38694.0


In [15]:
df_crime_socio = (
df_crime_socio.withColumn("month", func.month(func.col("date"))).
withColumn("year", func.year(func.col("date"))).
withColumn("day", func.dayofmonth(func.col("date"))).
withColumn("hour", func.hour(func.col("date"))).withColumn("minute", func.minute(func.col("date"))).
withColumn("dayofmonth", func.dayofmonth(func.col("date"))).   
withColumn("dayofyear", func.dayofyear(func.col("date"))).
withColumn("dayofweek", func.dayofweek(func.col("date")))
)

In [62]:
df_crime_socio.limit(100).toPandas().sample(5)

,community_area_number,id,cas_number,date,block,iucr,primary_type,description,location_description,arrest,...,isBROADWAY,isPKWY,duree_day,month,day,hour,minute,dayofmonth,dayofyear,dayofweek
4,27,9394705,HW538170,2013-11-18 12:38:00,029XX W VAN BUREN ST,143A,WEAPONS VIOLATION,UNLAWFUL POSS OF HANDGUN,STREET,true,...,0,0,35012.0,11,18,12,38,18,322,2
88,68,10039684,HY229255,2013-01-31 00:00:00,075XX S UNION AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,false,...,0,0,35967.0,1,31,0,0,31,31,5
62,44,10019848,HY208916,2013-08-01 00:01:00,079XX S ST LAWRENCE AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,false,...,0,0,51920.0,8,1,0,1,1,213,5
8,42,10026833,HY216282,2013-04-15 00:00:00,061XX S GREENWOOD AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,true,...,0,0,47998.0,4,15,0,0,15,105,2
77,24,10030520,HY220427,2013-10-15 15:00:00,014XX W NORTH AVE,0810,THEFT_ROBBERY_BURGLARY,OVER $500,OTHER,false,...,0,0,39957.0,10,15,15,0,15,288,3


In [63]:
df_crime_socio.printSchema()

root
 |-- community_area_number: string (nullable = true)
 |-- id: string (nullable = true)
 |-- cas_number: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- block: string (nullable = true)
 |-- iucr: string (nullable = true)
 |-- primary_type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location_description: string (nullable = true)
 |-- arrest: string (nullable = true)
 |-- domestic: string (nullable = true)
 |-- beat: string (nullable = true)
 |-- district: string (nullable = true)
 |-- ward: string (nullable = true)
 |-- fbi_code: string (nullable = true)
 |-- x_coordinate: string (nullable = true)
 |-- y_coordinate: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- updated_on: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- location: string (nullable = true)
 |-- community_area_name: string (nullable = true)
 |-- pct_housing_crowded: float (nullable = tr

In [64]:
df_temp.show(5)

+-----------+-------------------+-----+----+---+----+
|Temperature|           datetime|month|year|day|hour|
+-----------+-------------------+-----+----+---+----+
|     273.43|2013-01-01 01:00:00|    1|2013|  1|   1|
|     273.48|2013-01-01 02:00:00|    1|2013|  1|   2|
|     273.27|2013-01-01 03:00:00|    1|2013|  1|   3|
|     273.19|2013-01-01 04:00:00|    1|2013|  1|   4|
|     273.11|2013-01-01 05:00:00|    1|2013|  1|   5|
+-----------+-------------------+-----+----+---+----+
only showing top 5 rows



In [65]:
df_temp = df_temp.drop('datetime')

In [66]:
df_total = df_crime_socio.join(df_temp, ['year', 'month','day','hour'], how = "inner").\
                        join(df_sky, ['year', 'month','day','hour'], how = "inner")

In [67]:
df_total = df_total.na.drop()

In [68]:
df_total.limit(1000).toPandas().sample(5).T

,941,860,410,539,188
year,2013,2013,2013,2013,2013
month,1,1,3,7,7
day,1,1,6,4,26
hour,6,4,21,21,12
community_area_number,23,30,23,67,73
id,8951170,8951035,20819,20964,10149449
cas_number,HW100411,HW100332,HW181865,HW348220,HY338883
date,2013-01-01 06:15:00,2013-01-01 04:20:00,2013-03-06 21:15:00,2013-07-04 21:38:00,2013-07-26 12:00:00
block,043XX W CHICAGO AVE,026XX S SAWYER AVE,013XX N ST LOUIS AVE,062XX S MARSHFIELD AVE,090XX S MORGAN ST
iucr,1320,0486,0110,0110,1153


In [69]:
df_total = df_total.withColumn('latitude', df_total['latitude'].cast(FloatType()))
df_total = df_total.withColumn('longitude', df_total['longitude'].cast(FloatType()))

In [70]:
list_of_crimes = config["List_of_crimes_prediction"]["with_merge_pred"]

In [72]:
df = df_total.where(col('primary_type').isin(list_of_crimes))

In [76]:
df_ml = df.sample(0.1)

In [77]:
categoricalColumns = ['domestic']
numericCols = ['year', 'month', 'day', 'hour','minute', 'latitude',
               'longitude', 'isStreet', 'isAV',
               'isBLVD', 'isRD', 'isPL', 'isBROADWAY',
               'isPKWY', 'duree_day', 'minute','dayofmonth',
               'dayofyear','dayofweek','Temperature',
               'pct_housing_crowded',
               'pct_households_below_poverty',
               'pct_age16_unemployed',
               'pct_age25_no_highschool',
               'pct_not_working_age',
               'per_capita_income',
               'hardship_index',
                'Chicago_broken clouds',
                'Chicago_drizzle',
                 'Chicago_few clouds',
                'Chicago_fog',
                 'Chicago_haze',
                 'Chicago_heavy intensity drizzle',
                 'Chicago_heavy intensity rain',
                 'Chicago_heavy snow',
                 'Chicago_light intensity drizzle',
                 'Chicago_light rain',
                 'Chicago_light rain and snow',
                 'Chicago_light snow',
                 'Chicago_mist',
                 'Chicago_moderate rain',
                 'Chicago_overcast clouds',
                 'Chicago_proximity thunderstorm',
                 'Chicago_scattered clouds',
                 'Chicago_sky is clear',
                 'Chicago_snow',
                 'Chicago_thunderstorm',
                 'Chicago_thunderstorm with heavy rain',
                 'Chicago_thunderstorm with light rain',
                 'Chicago_thunderstorm with rain',
                'Chicago_very heavy rain'
]


In [123]:
rf = RandomForestClassifier(labelCol='label', featuresCol='features',numTrees=200)

In [124]:
stages = []
label_stringIdx = StringIndexer(inputCol = 'primary_type', outputCol = 'label')
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages.append(stringIndexer)
    stages.append(encoder)
stages.append(label_stringIdx)
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages.append(assembler)
stages.append(rf)

In [125]:
stages

[StringIndexer_722ebf1569d2,
 OneHotEncoderEstimator_f4bc19494f50,
 StringIndexer_f72ade94f7b5,
 VectorAssembler_5e3e0678834a,
 RandomForestClassifier_8e146858ac93]

In [126]:
df_train, df_test = df.randomSplit([0.7, 0.3])

In [127]:
pipeline_and_model = Pipeline(stages = stages)

In [ ]:
pipeline_model = pipeline_and_model.fit(df_train)

In [ ]:
df_test_pred = pipeline_model.transform(df_test)